In [ ]:
from datetime import datetime

from lmsiage.uncertainty import ComputeIntegratedUncertainty
from lmsiage.mesh_file import MeshFile
from lmsiage.zarr_index_tools import cleanup_missing_files, files_with_arrays, update_index_for_dir

In [ ]:
n_steps = 2210  # should be 2210 to cover 6 years
sia_dir = 'NERSC_arctic25km_sea_ice_age_v2p1/zarr'
mesh_dir = f'{sia_dir}/mesh'
age_dir = f'{sia_dir}/age'
unc_dir = f'{sia_dir}/uncert'

In [ ]:
update_index_for_dir(age_dir)
update_index_for_dir(unc_dir)

In [ ]:
unc_files = files_with_arrays(['unc_sic', 'unc_sid'])
unc_dates = [datetime.strptime(f.split('/')[-1], 'unc_%Y%m%d.zip') for f in unc_files]
start_indices = [i for i in range(0, len(unc_dates)) if unc_dates[i].month == 9 and unc_dates[i].day == 15]
start_indices

In [ ]:
ciu = ComputeIntegratedUncertainty(mesh_dir, age_dir, unc_dir, n_steps)
for start_idx in start_indices:
    ciu.compute(unc_dates, start_idx)


In [ ]:
year = 1999
unc_file = f'NERSC_arctic25km_sea_ice_age_v2p1/zarr/uncert/{year}/unc_{year}1231.zip'
mesh_file = f'NERSC_arctic25km_sea_ice_age_v2p1/zarr/mesh/{year}/mesh_{year}1231.zip'
print(MeshFile(unc_file).read_names())

import matplotlib.pyplot as plt
unc_sic, unc_sid = MeshFile(unc_file).load([f'unc_sic{year}', f'unc_sid{year}'], as_dict=False)
x, y, t = MeshFile(mesh_file).load(['x', 'y', 't'], as_dict=False)
plt.tripcolor(x, y, t, unc_sid)
plt.colorbar()
plt.show()

plt.tripcolor(x, y, t, unc_sic)
plt.colorbar()
plt.show()